In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 11830665
paper_name = 'fleming_blackman_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['hit_data.txt','hit_data2.txt']

In [64]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/' + f, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['gene'] = original_data['genenames'].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data.set_index('orf', inplace=True)
    
    if ixf == 1:
        data_cols = ['ps-519','ps-341']
        for c in data_cols:
            original_data[c] = 1
    else:
        data_cols = ['ps-519','ps-341']
        original_data[data_cols] = original_data[data_cols].apply(pd.to_numeric, axis=1, errors='coerce')
        
        wt = original_data.loc['WT',data_cols]
        
        original_data[data_cols] = original_data[data_cols].subtract(wt, axis=1).divide(wt, axis=1)
        original_data.drop(index='WT', inplace=True)
        
    original_data = original_data[data_cols].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 53 x 3
  genenames  ps-341  ps-519
0        wt    1300    1200
1      rpn4      50      10
2     rpn10     200      40
3      pre9     650     150
4     apg17     250     120
            genenames  ps-341  ps-519 gene orf
index_input                                   
0                  wt    1300    1200   WT  WT
(52, 2)
Original data dimensions: 7 x 1
  genenames
0   ydr136c
1      ypt6
2      ric1
3      bem4
4      yme1
Empty DataFrame
Columns: [genenames, gene, orf]
Index: []
(7, 2)


In [65]:
original_data = pd.concat(original_data_list, axis=0)

In [66]:
original_data.shape

(59, 2)

In [67]:
original_data = original_data.groupby(original_data.index).mean()

In [68]:
original_data.shape

(59, 2)

# Load & process tested strains

In [69]:
tested = pd.read_excel('raw_data/Table 2 (web supplement).xlsx', sheet_name='Sheet1', skiprows=4)

In [70]:
tested.head()

,ORF Name,Gene Name,Genomics,1 hr,2 hr,4 hr,1 hr.1,2 hr.1,4 hr.1,1 hr.2,...,2 hr.7,4 hr.7,2 hr.8,4 hr.8,1 hr.6,2 hr.9,4 hr.9,1 hr.7,2 hr.10,4 hr.10
0,YAL001C,TFC3,NaN,0.832,0.967,0.966,0.886,1.129,0.654,1.061,...,0.960,1.413,0.817,1.381,0.958,1.155,1.083,1.121,1.068,0.838
1,YAL002W,VPS8,+,0.830,0.878,0.946,1.084,1.485,1.157,1.149,...,1.176,1.261,1.085,1.310,1.053,1.013,1.069,1.041,1.091,1.039
2,YAL003W,EFB1,NaN,0.931,0.894,0.969,0.932,0.796,0.982,0.724,...,0.900,1.268,0.832,0.918,0.943,0.806,0.987,0.729,0.726,1.038
3,YAL004W,YAL004W,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YAL005C,SSA1,+,1.544,1.199,1.037,2.478,0.871,2.135,2.974,...,1.209,1.228,1.968,1.638,1.387,1.749,1.843,2.119,1.676,2.093


In [71]:
tested.shape

(6086, 33)

In [72]:
tested = tested.loc[tested['Genomics']=='+',:]
tested.shape

(4276, 33)

In [73]:
tested['orf'] = tested['ORF Name'].astype(str)

In [74]:
tested['orf'] = clean_orf(tested['orf'])

In [75]:
# Insert missing '-'
tested['orf'] = tested['orf'].apply(lambda x: x[:7] + '-' + x[7] if len(x) == 8 else x)

In [76]:
tested['orf'] = translate_sc(tested['orf'].values, to='orf')

In [77]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF Name, Gene Name, Genomics, 1 hr, 2 hr, 4 hr, 1 hr.1, 2 hr.1, 4 hr.1, 1 hr.2, 2 hr.2, 4 hr.2, 1 hr.3, 2 hr.3, 4 hr.3, 1 hr.4, 2 hr.4, 4 hr.4, 1 hr.5, 2 hr.5, 4 hr.5, 2 hr.6, 4 hr.6, 2 hr.7, 4 hr.7, 2 hr.8, 4 hr.8, 1 hr.6, 2 hr.9, 4 hr.9, 1 hr.7, 2 hr.10, 4 hr.10, orf]
Index: []

[0 rows x 34 columns]


In [78]:
tested_orfs = tested['orf'].unique()

In [79]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YDR300C', 'YLR423C']

In [80]:
tested_orfs = list(tested_orfs) + missing

In [81]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [82]:
data = original_data.copy()

In [83]:
dataset_ids = [1308, 471]
datasets = datasets.reindex(index=dataset_ids)

In [84]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [85]:
data.head()

dataset_id,1308,471
data_type,value,value
orf,,
YAL002W,0.0,0.0
YAL004W,0.0,0.0
YAL005C,0.0,0.0
YAL007C,0.0,0.0
YAL008W,0.0,0.0


## Subset to the genes currently in SGD

In [86]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 18


In [87]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1308,471
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.0,0.0
1863,YAL004W,0.0,0.0
4,YAL005C,0.0,0.0
5,YAL007C,0.0,0.0
6,YAL008W,0.0,0.0


# Normalize

In [88]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [89]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [90]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,1308,471,1308,471
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.0,0.0,0.059668,-0.108176
1863,YAL004W,0.0,0.0,0.059668,-0.108176
4,YAL005C,0.0,0.0,0.059668,-0.108176
5,YAL007C,0.0,0.0,0.059668,-0.108176
6,YAL008W,0.0,0.0,0.059668,-0.108176


# Print out

In [91]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [92]:
from IO.save_data_to_db3 import *

In [93]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 11830665...
Inserting the new data...


100%|██████████| 2/2 [00:13<00:00,  6.77s/it]

Updating the data_modified_on field...
